In [9]:
import pandas as pd
import pickle


In [10]:
# Need to import pickled logistic classsifiers and data scaler from the training dataset
with open('models/scaler.p', 'rb') as fp:
     scaler = pickle.load(fp)
# 
with open('models/2sogs.p', 'rb') as fp:
    two_sogs_model = pickle.load(fp)

with open('models/3sogs.p', 'rb') as fp:
    three_sogs_model = pickle.load(fp)

with open('models/4sogs.p', 'rb') as fp:
    four_sogs_model = pickle.load(fp)

with open('models/5sogs.p', 'rb') as fp:
    five_sogs_model = pickle.load(fp)

In [11]:
most_recent_sog_data = pd.read_csv('nhl_data/current_sogs_inputs.csv')
most_recent_sog_data.head()

,Unnamed: 0,Player ID,Player Name,SoGs,Date,sogs_calc,Trailing 1 Game Avg SoGs,Trailing 3 Game Avg SoGs,Trailing 5 Game Avg SoGs,Trailing 7 Game Avg SoGs,...,Prediction 1 Game Avg SoGs,Prediction 3 Game Avg SoGs,Prediction 5 Game Avg SoGs,Prediction 7 Game Avg SoGs,Prediction 10 Game Avg SoGs,Prediction Average SoGs,Over 1.5,Over 2.5,Over 3.5,Over 4.5
0,8481600,8481600,T. Bjornfot,0,2023-10-11,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0,0,0,0
1,8481523,8481523,K. Dach,0,2023-10-14,3.0,3.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,1.500000,0,0,0,0
2,8483489,8483489,F. Minten,1,2023-10-19,1.0,1.0,0.666667,0.0,0.0,...,1.0,0.666667,0.0,0.0,0.0,0.750000,0,0,0,0
3,8482088,8482088,J. Sourdif,1,2023-10-21,0.0,0.0,0.000000,0.0,0.0,...,1.0,0.666667,0.0,0.0,0.0,0.666667,0,0,0,0
4,8482250,8482250,C. Burke,0,2023-10-21,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0,0,0,0


In [12]:
# Create a function that accepts a player and opposing team and returns an X Value ready for prediction
def get_x_val(player_name):
    df_sogs = pd.read_csv('nhl_data/current_sogs_inputs.csv')
    df_player_x_val = df_sogs[df_sogs['Player Name']==player_name]

    df_player_x_val = df_player_x_val.loc[:,['Prediction Average SoGs', 'Prediction 1 Game Avg SoGs', 'Prediction 3 Game Avg SoGs', 'Prediction 5 Game Avg SoGs', 'Prediction 10 Game Avg SoGs']]
    
    # Rename the columns to be in line with the scaler
    df_player_x_val = df_player_x_val.rename(columns={'Prediction Average SoGs': "Average SoGs"
                                                      , "Prediction 1 Game Avg SoGs": "Trailing 1 Game Avg SoGs"
                                                      , "Prediction 3 Game Avg SoGs": "Trailing 3 Game Avg SoGs"
                                                      , "Prediction 5 Game Avg SoGs": "Trailing 5 Game Avg SoGs"
                                                      , "Prediction 10 Game Avg SoGs": "Trailing 10 Game Avg SoGs"})
    

    return df_player_x_val


In [13]:
def make_sog_prediction(player, scal, model):
    player_x_val = get_x_val(player)
    player_x_val = scal.transform(player_x_val)
    player_pred = model.predict(player_x_val)
    #print(f"Over 2.5 shots for {player}:{player_pred}")
    return player_pred

In [14]:
list_all_players = list(most_recent_sog_data.loc[:, 'Player Name'])
list_all_players_avgs = list(most_recent_sog_data.loc[:, "Prediction Average SoGs"])

for player, avg in zip(list_all_players, list_all_players_avgs):
    three_sogs_pred = make_sog_prediction(player, scaler, three_sogs_model)
    if len(three_sogs_pred) == 1:
        if three_sogs_pred == 1 and avg <= 3.5:
            print(f"Over 2.5 shots for {player}:{three_sogs_pred}. Their avg sogs is {avg}")



Over 2.5 shots for B. Rust:[1]. Their avg sogs is 3.227272727272727
Over 2.5 shots for T. Konecny:[1]. Their avg sogs is 3.242424242424242
Over 2.5 shots for O. Tippett:[1]. Their avg sogs is 3.4545454545454546
Over 2.5 shots for V. Nichushkin:[1]. Their avg sogs is 3.03125
Over 2.5 shots for D. Larkin:[1]. Their avg sogs is 3.4642857142857144
Over 2.5 shots for E. Pettersson:[1]. Their avg sogs is 2.685714285714285
Over 2.5 shots for A. DeBrincat:[1]. Their avg sogs is 3.264705882352941
Over 2.5 shots for A. Barkov:[1]. Their avg sogs is 3.1
Over 2.5 shots for C. Bedard:[1]. Their avg sogs is 3.1818181818181817
Over 2.5 shots for T. Moore:[1]. Their avg sogs is 3.3
Over 2.5 shots for F. Vatrano:[1]. Their avg sogs is 3.4545454545454546
Over 2.5 shots for J. Guentzel:[1]. Their avg sogs is 3.34375
Over 2.5 shots for J. Robertson:[1]. Their avg sogs is 2.78125
Over 2.5 shots for N. Kadri:[1]. Their avg sogs is 3.2058823529411766
Over 2.5 shots for P. Kane:[1]. Their avg sogs is 3.5
Over

In [15]:
for player, avg in zip(list_all_players, list_all_players_avgs):
    four_sogs_pred = make_sog_prediction(player, scaler, four_sogs_model)
    if len(four_sogs_pred) == 1:
        if four_sogs_pred == 1 and avg <= 8:
            print(f"Over 3.5 shots for {player}:{four_sogs_pred}. Their avg sogs is {avg}")


Over 3.5 shots for A. Matthews:[1]. Their avg sogs is 4.366666666666666
Over 3.5 shots for J. Eichel:[1]. Their avg sogs is 4.2
Over 3.5 shots for J. Tavares:[1]. Their avg sogs is 3.806451612903226
Over 3.5 shots for W. Nylander:[1]. Their avg sogs is 4.32258064516129
Over 3.5 shots for A. Panarin:[1]. Their avg sogs is 4.1875
Over 3.5 shots for A. Ovechkin:[1]. Their avg sogs is 3.838709677419355
Over 3.5 shots for D. Pastrnak:[1]. Their avg sogs is 5.09375
Over 3.5 shots for N. Kucherov:[1]. Their avg sogs is 4.588235294117647
Over 3.5 shots for B. Tkachuk:[1]. Their avg sogs is 4.724137931034483
Over 3.5 shots for N. MacKinnon:[1]. Their avg sogs is 4.5
Over 3.5 shots for J. Hughes:[1]. Their avg sogs is 4.444444444444445


In [16]:
for player, avg in zip(list_all_players, list_all_players_avgs):
    five_sogs_pred = make_sog_prediction(player, scaler, five_sogs_model)
    if len(five_sogs_pred) == 1:
        if five_sogs_pred == 1:
            print(f"Over 4.5 shots for {player}:{five_sogs_pred}. Their avg sogs is {avg}")


Over 4.5 shots for A. Matthews:[1]. Their avg sogs is 4.366666666666666
Over 4.5 shots for J. Eichel:[1]. Their avg sogs is 4.2
Over 4.5 shots for J. Tavares:[1]. Their avg sogs is 3.806451612903226
Over 4.5 shots for W. Nylander:[1]. Their avg sogs is 4.32258064516129
Over 4.5 shots for A. Panarin:[1]. Their avg sogs is 4.1875
Over 4.5 shots for A. Ovechkin:[1]. Their avg sogs is 3.838709677419355
Over 4.5 shots for D. Pastrnak:[1]. Their avg sogs is 5.09375
Over 4.5 shots for N. Kucherov:[1]. Their avg sogs is 4.588235294117647
Over 4.5 shots for B. Tkachuk:[1]. Their avg sogs is 4.724137931034483
Over 4.5 shots for N. MacKinnon:[1]. Their avg sogs is 4.5
Over 4.5 shots for J. Hughes:[1]. Their avg sogs is 4.444444444444445


In [17]:
list_two_sogs = []
for player, avg in zip(list_all_players, list_all_players_avgs):
    two_sogs_pred = make_sog_prediction(player, scaler, two_sogs_model)
    if len(two_sogs_pred) == 1:
        if two_sogs_pred == 1 and avg <= 2.5:
            print(f"Over 1.5 shots for {player}:{two_sogs_pred}. Their avg sogs is {avg}")
            list_two_sogs.append(player)

two_sogs = pd.DataFrame(data={"Player Name":list_two_sogs})
two_sogs.to_csv("two_sogs.csv")

Over 1.5 shots for F. Chytil:[1]. Their avg sogs is 2.2
Over 1.5 shots for A. Athanasiou:[1]. Their avg sogs is 1.818181818181818
Over 1.5 shots for J. Roslovic:[1]. Their avg sogs is 2.071428571428572
Over 1.5 shots for B. Hayton:[1]. Their avg sogs is 2.125
Over 1.5 shots for T. Hall:[1]. Their avg sogs is 2.0
Over 1.5 shots for J. Schwartz:[1]. Their avg sogs is 2.260869565217391
Over 1.5 shots for M. Jankowski:[1]. Their avg sogs is 2.5
Over 1.5 shots for A. Nylander:[1]. Their avg sogs is 2.0
Over 1.5 shots for A. Burakovsky:[1]. Their avg sogs is 2.142857142857143
Over 1.5 shots for J. Brodin:[1]. Their avg sogs is 1.92
Over 1.5 shots for J. Vrana:[1]. Their avg sogs is 2.1578947368421053
Over 1.5 shots for T. Pearson:[1]. Their avg sogs is 1.7407407407407407
Over 1.5 shots for M. Coronato:[1]. Their avg sogs is 2.1818181818181817
Over 1.5 shots for S. Jones:[1]. Their avg sogs is 2.2222222222222223
Over 1.5 shots for P. Laine:[1]. Their avg sogs is 2.2777777777777777
Over 1.5 sh

In [18]:
list_players = ['M. Bunting',
                'G. Forsling',
                'N. Paul',
                'L. Draisaitl',
                'C. Bedard',
                'A. DeBrincat',
                'C. Atkinson',
                'A. Ovechkin',
                'M. Zibanejad',
                'C. Kreider',
                'T. Hertl',
                'N. Dobson']
for pl in list_players:
    print(make_sog_prediction(pl, scaler, three_sogs_model))


[0]
[0]
[0]
[0]
[1]
[1]
[0]
[1]
[0]
[0]
[0]
[0]


In [19]:
most_recent_sog_data[most_recent_sog_data['Player Name'] == "C. Kreider"]


,Unnamed: 0,Player ID,Player Name,SoGs,Date,sogs_calc,Trailing 1 Game Avg SoGs,Trailing 3 Game Avg SoGs,Trailing 5 Game Avg SoGs,Trailing 7 Game Avg SoGs,...,Prediction 1 Game Avg SoGs,Prediction 3 Game Avg SoGs,Prediction 5 Game Avg SoGs,Prediction 7 Game Avg SoGs,Prediction 10 Game Avg SoGs,Prediction Average SoGs,Over 1.5,Over 2.5,Over 3.5,Over 4.5
690,8475184,8475184,C. Kreider,4,2023-12-23,1.0,1.0,1.666667,2.8,3.142857,...,4.0,1.666667,3.0,3.428571,2.7,2.65625,1,1,1,0


# Model Picks
### All picks are one unit and on o 2.5 SoGs unless noted otherwise

Model: 6-3 (+1.8u)

5/11 McCann Win -135 (+.74) W
5/12 Barkov Win -130 (+.77), W
5/16 McCann Loss o 2.5 -118 (-1), L
5/18 Barkov -110 (+.9) W
5/18 Verhaeghe -110 (+.9) W
5/20 Barkov -118 (+.8) W
5/20 Verhaeghe -110 (-1) L
5/24 Verhaeghe -130 (-1) L 
5/25 Domi o1.5  -135 (+.74) W
5/27 Domi o1.5 -135 (-1) L
5/29 Domi o1.5 -152 (+.6) W 
6/8 Duclair o1.5 -180 (-.5)L
6/8 Smith  o1.5 -180 (-.5)L
6/10 Smith o1.5 -155 (+.6) W


Model 2023-24 Season (9-9) -2.4u
12/4 Atkinson o2.5 -144 Win .7
12/5 Stutzle 02.5 -140 Loss -1
12/6 Paul o2.5 -135 Loss -1
12/7 Paul o2.5 -115 Loss -1
12/8 R Thomas -150 Win .7
12/10 A Kempe -160 Loss -1
12/11 P Kane -135 Loss -1
12/12 C Bedard  -152 Loss -1
12/14 Papi o4.5 -118 Win +.9
12/14 A DeBrincat o2.5 -145  Win +.7
12/15 Barzal -145 Loss -1
12/15 Timmy Stu -140 win +.7
12/16 T Moore -130 win  +.8
12/16 Atkinson -125  Loss -1
12/17 Eichel o3.5 -135 Win +.7
12/18 Barkov o2.5 -122 Win +.8 
12/19 B. Nelson -135 Win +.7
12/19 C. Atkinson o2.5 +105 Loss -1
12/20 D Larkin -150 o2.5 Win +.7
12/21 C Keller o2.5 -140 Win +.7
12/21 J Eichel o3.5 -130 Win +.8
12/21 A Matthews o4.5 +110 Loss -1
12/21 E Bouchard o2.5 -115 Loss -1
12/27 W Nylander o3.5 -140
12/27 J Eichel o3.5 -140
12/27 A Barkov o2.5 -135
12/27 R Josi o2.5 -140


